<a href="https://colab.research.google.com/github/jaunario/atmos/blob/main/AgERA5_data_for_ORYZA_Simplified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTRUCTIONS
---
1. Open the ***Table of Contents*** on the left by clickin on the 1st button on the upper left side of the page just below the Google Colab logo, which looks like am orange infinity symbol.

2. On the *Table of Contents* point your mouse on ***Prepare the runtime environment*** and click on the vertically-aligned 3 dots and click ***Run cells in this section***, and you will see at the bottom of the section that a circle with a square inside will start spinning. Wait for about 10-15mins until you see the spinning stopped.

3. Specify the ISO3 code for the country of your area of interest (AOI). When you're done, point your mouse over ***Area of Interest*** on the *Table of Contents*, click on the vertical 3 dots and click ***Run cells in section***. Inspect the generated map to verify that the AOI is correct.

4. Proceed to ***Generating the ORYZA Weather Files***.

In [ ]:
#@title Prepare the runtime environment
print("cdsapi package installation initiated.")

!pip install cdsapi --quiet
# Latest (as of 5 Jul 2023) rpy2 doesn't seem to work properly with rpy2.ipython.
# Until this has been resolved use 3.5.1

print("reverting rpy2 package version 3.5.1.")
!pip install rpy2==3.5.1 --quiet

%load_ext rpy2.ipython

import os
import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)


R_TEMP       = "./R_temp"
AGERA5_DLDIR = "./AgERA5"
EXTRACT_DIR  = "./extract"

UID = "220427"
Token = "c1601cc6-edff-418c-98d6-161fab1b000a"

if(not os.path.isdir(R_TEMP)):
  os.mkdir(R_TEMP)

if(not os.path.isdir(AGERA5_DLDIR)):
  os.mkdir(AGERA5_DLDIR)

print("Process folders ready.")

# Create authorization file for connecting to CDS
if(not os.path.isfile("/root/.cdsapirc")):
  file_cdsauth = open("/root/.cdsapirc", "x")
  file_cdsauth.writelines(["url: https://cds.climate.copernicus.eu/api/v2\n",
                         "key: " + UID + ":" + Token])
  file_cdsauth.close()

print("CDS Credentials ready.")

%R -i R_TEMP -i AGERA5_DLDIR -i EXTRACT_DIR

print("Installing R packages.")
!Rscript -e 'install.packages(c("terra", "geodata"), verbose=FALSE, repos="https://cloud.r-project.org")'

# Area of Interest

In [ ]:
#@title  { run: "auto" }
#@markdown *Administrative boundaries will be downloaded from [GADM](https://gadm.org/data.html) through the geodata package in R.*
#@markdown After running this cell, run
#@markdown **AOI Parameters.**

#@markdown ---


#@markdown **Country ISO3 code**
#@markdown  <br><sub>*(Don't know the code for your AOI? Search google or refer to [this list](https://docs.google.com/spreadsheets/d/1xdZSii55PPN5S3XaFOHBxpZDC_7n3b_HfBMN4wP3LDM/edit?usp=sharing))*
ISO3      = 'NGA' #@param {type:"string"}

OUTPUT_DIR   = "./fse/" + ISO3

%R -i OUTPUT_DIR

#@markdown **Administrative Level** <br><sub>*(0 = Country Level, 3 = Smallest available admin level)*
ADM_LEVEL = "0" #@param ["0", "1", "2", "3"]
ADM_LEVEL = int(ADM_LEVEL)

#@markdown **Name of Subregion**
#@markdown <br><sub>*(If you need more specific regions. Separate by comma. Must be listed in the correct admin level, and spelling matches with GADM)*
FILTER    = "" #@param {type:"string"}

# These are inputs for specifying the Area of Intereset (AOI) and downloading GADM vector files.
#@markdown ---

# SUBNATIONAL
if ADM_LEVEL > 0:
  FILTER_FIELD = "NAME_" + str(ADM_LEVEL)
else:
  FILTER_FIELD = None
  FILTER       = None

if FILTER == "":
  ADM_LEVEL = 0
  FILTER_FIELD = None
  FILTER       = None

%R -i ISO3 -i ADM_LEVEL

if not FILTER_FIELD is None:
  %R -i FILTER_FIELD
if not FILTER is None:
  %R -i FILTER




In [ ]:
# @title Generating AOI object requirements from user parameters  { vertical-output: true }
#@markdown **Your AOI will show up below.** <br> <sub>Check the resulting map before proceeding

%%R

VARS = c("tmin", "tmax", "srad", "prec", "vp", "wind")
COPERNICUS_VARS = c("Temperature-Air-2m-Min-24h", "Temperature-Air-2m-Max-24h", "Solar-Radiation-Flux", "Precipitation-Flux", "Vapour-Pressure-Mean", "Wind-Speed-10m-Mean")

library(terra)
library(geodata)

rst.elev <- suppressMessages(geodata::elevation_global(res=5, path=R_TEMP))
vec_adm  <- suppressMessages(geodata::gadm(ISO3, level=ADM_LEVEL, path=R_TEMP))

if(exists("FILTER")) {
    dat.adm <- as.data.frame(vec_adm)
    #message(paste(match(trimws(FILTER), dat.adm[,FILTER_FIELD])), collapse=", "))
    idx.filter <- match(trimws(FILTER), dat.adm[,FILTER_FIELD])
    if(length(idx.filter>0)){
      vec_adm <- vec_adm[idx.filter,]
    } else {
      message("Check your list of filters as there are no features that matches them in ", FILTER_FIELD, ".")
      message("Spelling should be correct also.")
      message("DO NOT PROCEED. Until you se the map of your AOI is displayed.")
    }
}

vec_buff <- buffer(vec_adm, 11000)

area <- ext(vec_buff)
area <- as.vector(area)
area <- area[c(4,1,3,2)]
area <- round(area,1)

plot(vec_buff, main= paste("Area of Interest:", ISO3), col="grey")
plot(vec_adm, border="blue", col="grey", add=TRUE)



# Generating the ORYZA Weather Files
---
1. In ***Getting data from CDS***, specify the year of interest then click run colab_run.png.
2. Choose between <br>**Option 1: Native Resolution** (0.1 Degrees i.e., approximately 11 x 11 km). or <br> **Option 2: Resampled 0.25-deg Resolution** (approx 28 x 28 km). For RIICE-YES, use **Option 2**.

In [ ]:
# @title  Getting data from CDS { run: "auto" }
# @title {vertical-output: true, display-mode: "form" }
#@markdown ####Year of Interest (YOI)
#@markdown <sub> *(Starts from 1979 up to the curremt year. Keep in mind that of the 1 month lag if you're downloading the current year)*

YEAR = 2023 #@param {type:"integer"}

from os.path import isfile


%R -o area
area = area.tolist()

import cdsapi

client = cdsapi.Client()
if(not os.path.isfile(AGERA5_DLDIR + "/temp_" + str(YEAR) + "_" + ISO3 +".zip")):
  temp = client.retrieve('sis-agrometeorological-indicators',
    {
       'variable': ['2m_temperature'],
       'statistic':  ['24_hour_minimum', '24_hour_maximum'],
       'year': str(YEAR),
       'month': [
           '01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12'
       ],
       'day': [
           '01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12',
           '13', '14', '15', '16', '17', '18',
           '19', '20', '21', '22', '23', '24',
           '25', '26', '27', '28', '29', '30',
           '31'
      ],
      'area': area
    },
    AGERA5_DLDIR + "/temp_" + str(YEAR) + "_" + ISO3 +".zip")

if(not os.path.isfile(AGERA5_DLDIR + "/vapr_" + str(YEAR) + "_" + ISO3 +".zip")):
  vapr = client.retrieve('sis-agrometeorological-indicators',
    {
       'variable': 'vapour_pressure',
       'statistic':  '24_hour_mean',
       'year': str(YEAR),
       'month': [
           '01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12'
       ],
       'day': [
           '01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12',
           '13', '14', '15', '16', '17', '18',
           '19', '20', '21', '22', '23', '24',
           '25', '26', '27', '28', '29', '30',
           '31'
        ],
        'area': area
    },
    AGERA5_DLDIR + "/vapr_" + str(YEAR) + "_" + ISO3 +".zip")

if(not os.path.isfile(AGERA5_DLDIR + "/wind_" + str(YEAR) + "_" + ISO3 +".zip")):
  wind = client.retrieve('sis-agrometeorological-indicators',
    {
        'variable': '10m_wind_speed',
        'statistic':  '24_hour_mean',
        'year': str(YEAR),
        'month': [
            '01', '02', '03', '04', '05', '06',
            '07', '08', '09', '10', '11', '12'
        ],
        'day': [
            '01', '02', '03', '04', '05', '06',
            '07', '08', '09', '10', '11', '12',
            '13', '14', '15', '16', '17', '18',
            '19', '20', '21', '22', '23', '24',
            '25', '26', '27', '28', '29', '30',
            '31'
        ],
        'area': area

    },
  AGERA5_DLDIR + "/wind_" + str(YEAR) + "_" + ISO3 +".zip")

if(not os.path.isfile(AGERA5_DLDIR + "/prec_" + str(YEAR) + "_" + ISO3 +".zip")):
  prec = client.retrieve('sis-agrometeorological-indicators',
    {
       'variable': 'precipitation_flux',
       'year': str(YEAR),
       'month': [
           '01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12'
       ],
       'day': [
           '01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12',
           '13', '14', '15', '16', '17', '18',
           '19', '20', '21', '22', '23', '24',
           '25', '26', '27', '28', '29', '30',
           '31'
       ],
       'area': area
    },
  AGERA5_DLDIR + "/prec_" + str(YEAR) + "_" + ISO3 +".zip")

if(not os.path.isfile(AGERA5_DLDIR + "/srad_" + str(YEAR) + "_" + ISO3 +".zip")):
  srad = client.retrieve('sis-agrometeorological-indicators',
    {
      'variable': 'solar_radiation_flux',
      'year': str(YEAR),
      'month': [
          '01', '02', '03', '04', '05', '06',
          '07', '08', '09', '10', '11', '12'
      ],
      'day': [
          '01', '02', '03', '04', '05', '06',
          '07', '08', '09', '10', '11', '12',
          '13', '14', '15', '16', '17', '18',
          '19', '20', '21', '22', '23', '24',
          '25', '26', '27', '28', '29', '30',
          '31'
      ],
      'area': area
    },
      AGERA5_DLDIR + "/srad_" + str(YEAR) + "_" + ISO3 +".zip")



from zipfile import ZipFile
from os import listdir

files_zipped =  listdir(AGERA5_DLDIR)
files_zipped =  [fz for fz in files_zipped if str(YEAR)+"_"+ISO3 in fz]

if(os.path.isdir(EXTRACT_DIR)):
  files_unzipped =  listdir(EXTRACT_DIR)
  os.chdir(EXTRACT_DIR)
  print("Deleting " + str(len(files_unzipped)) + " old files.")
  for filez in files_unzipped:
    os.remove(filez)
  os.chdir("..")
  os.rmdir(EXTRACT_DIR)
  os.mkdir(EXTRACT_DIR)
else:
  os.mkdir(EXTRACT_DIR)

for filez in files_zipped:
  with ZipFile(AGERA5_DLDIR + "/" + filez) as zipo:
    zipo.extractall(path=EXTRACT_DIR)


%R -i YEAR


In [ ]:
#@title Option 1: Native Resolution

%%R

nc.files <- dir(EXTRACT_DIR, pattern=".nc$", full.names=TRUE)
vals.container <- list()

rst.base <- rast()
res(rst.base) <- 0.1
values(rst.base) <- 1:ncell(rst.base)

rst.elev_6 <- terra::resample(rst.elev, rst.base)

rst.base <- mask(rst.base, rst.elev_6)
rst.base <- crop(rst.base, vec_buff)

rst.aoi <- rasterize(vec_buff, rst.base)
rst.aoi <- crop(rst.aoi, rst.base)
rst.aoi <- mask(rst.aoi, rst.base)

rst.aoicells <- crop(rst.base, rst.aoi)
rst.aoicells <- mask(rst.aoicells, rst.aoi)
idx <- which(!is.na(values(rst.aoi)))
cells <- rst.aoicells[idx]

elev <- rst.elev_6[cells[,1]]
elev[is.na(elev)] <- -99


for (i in 1:length(VARS)){
  var.files <- grep(COPERNICUS_VARS[i], nc.files,value=TRUE)
  cat(VARS[i], ": Loading ", length(var.files), " rasters \n")
  rst.var <- rast(var.files)
  dates.avail <- time(rst.var)
  rst.var <- resample(rst.var, rst.aoi, method="near")
  rst.var <- mask(rst.var, rst.aoi)

  rst.var <- switch(VARS[i], srad=rst.var/1000, tmax=rst.var-273.15, tmin=rst.var-273.15, tavg=rst.var-273.15, vp=rst.var/10, rst.var)
  vals.container[[i]] <- as.matrix(rst.var[idx])
  colnames(vals.container[[i]]) <- format(dates.avail,"D%Y%j")
}
names(vals.container) <- VARS

if(dir.exists(OUTPUT_DIR)){
    unlink(OUTPUT_DIR, recursive=TRUE)
}
dir.create(OUTPUT_DIR, recursive=TRUE)

dates.thisyear <- seq(as.Date(paste0("1/1/",YEAR), "%m/%d/%Y"), as.Date(paste0("12/31/",YEAR), "%m/%d/%Y"), by="day")
files.out <- vector()

prog.x <- 2
cat("Creating FSE files: |")
for(i in 1:length(idx)){
  this.cell <- data.frame(cell= cells[i,1], year=YEAR, doy=as.numeric(format(dates.thisyear, "%j")))
  wth <- vector()
  for(j in 1:length(VARS)){
      #message(dim(vals.container[[j]]))
      dates.avail <- as.Date(colnames(vals.container[[j]]), "D%Y%j")
      if(VARS[j]=="srad") vals <- round(vals.container[[j]][i,],0) else vals <- round(vals.container[[j]][i,],1)
      vals[is.na(vals)] <- -99
      this.cell[,VARS[j]] <- -99
      this.cell[match(dates.avail, dates.thisyear), VARS[j]] <- vals
  }
  this.cell <- this.cell[,c("cell", "year", "doy", "srad", "tmin", "tmax", "vp", "wind", "prec")]
  chk.vals <- colMeans(this.cell[,c("srad", "tmin", "tmax", "vp", "wind", "prec")])
  if(sum(chk.vals==-99)==6) next
  to.file <- c(paste(t(c(xyFromCell(rst.aoi, idx[i]), round(elev[i,1],1), -99, -99)), collapse=", "),
                apply(this.cell, 1, paste, collapse=", "))

  writeLines(to.file, con=paste0(OUTPUT_DIR,"/nsch", cells[i,1], ".", substr(YEAR,2,4)))
  files.out <- c(files.out,paste0(OUTPUT_DIR,"/nsch", cells[i,1], ".", substr(YEAR,2,4)))

  progress <- i/length(idx)*100
  if(progress>=prog.x) {
      cat("=")
      prog.x <- prog.x + 2
  }
}
cat("| Done. \n")

zip(paste0(OUTPUT_DIR,"/../", paste(ISO3, YEAR, "6arcmin", sep="_"), ".zip"), files.out, flags= "-j -r9X")

cat("Download the output \n", paste0(sub(ISO3, "", OUTPUT_DIR), paste(ISO3, YEAR, "6arcmin", sep="_"), ".zip"))

In [ ]:
#@title Option 2: Resampled 0.25-deg Resolution

# @markdown *RIICE-YES, to this date uses 0.25 resolution data so run this section until advised to use native-resolution of AgERA5.*

%%R

rst.elev_15 <- terra::aggregate(rst.elev, fact=3, na.rm=TRUE)

rst.base <- rast(rst.elev_15)
values(rst.base) <- 1:ncell(rst.base)
rst.base <- mask(rst.base, rst.elev_15)
rst.base <- crop(rst.base, vec_buff)

rst.aoi <- rasterize(vec_buff, rst.base)
rst.aoi <- crop(rst.aoi, rst.base)
rst.aoi <- mask(rst.aoi, rst.base)

rst.aoicells <- crop(rst.base, rst.aoi)
rst.aoicells <- mask(rst.aoicells, rst.aoi)
idx <- which(!is.na(values(rst.aoi)))
cells <- rst.aoicells[idx]

elev <- rst.elev_15[cells[,1]]
elev[is.na(elev)] <- -99

nc.files <- dir(EXTRACT_DIR, pattern=".nc$", full.names=TRUE)
vals.container <- list()

for (i in 1:length(VARS)){
  var.files <- grep(COPERNICUS_VARS[i], nc.files,value=TRUE)
  cat(VARS[i], ": Loading ", length(var.files), " rasters \n")
  rst.var <- rast(var.files)
  dates.avail <- time(rst.var)
  rst.var <- resample(rst.var, rst.aoi)
  rst.var <- mask(rst.var, rst.aoi)
  rst.var <- switch(VARS[i], srad=rst.var/1000, tmax=rst.var-273.15, tmin=rst.var-273.15, tavg=rst.var-273.15, vp=rst.var/10, rst.var)
  vals.container[[i]] <- as.matrix(rst.var[idx])
  colnames(vals.container[[i]]) <- format(dates.avail,"D%Y%j")
}
names(vals.container) <- VARS

if(dir.exists(OUTPUT_DIR)){
    unlink(OUTPUT_DIR, recursive=TRUE)
}
dir.create(OUTPUT_DIR, recursive=TRUE)

dates.thisyear <- seq(as.Date(paste0("1/1/",YEAR), "%m/%d/%Y"), as.Date(paste0("12/31/",YEAR), "%m/%d/%Y"), by="day")
files.out <- vector()
prog.x <- 2
cat("Creating FSE files: |")

for(i in 1:length(idx)){
  this.cell <- data.frame(cell= cells[i,1], year=YEAR, doy=as.numeric(format(dates.thisyear, "%j")))
  wth <- vector()
  for(j in 1:length(VARS)){
      #message(dim(vals.container[[j]]))
      dates.avail <- as.Date(colnames(vals.container[[j]]), "D%Y%j")
      if(VARS[j]=="srad") vals <- round(vals.container[[j]][i,],0) else vals <- round(vals.container[[j]][i,],1)
      vals[is.na(vals)] <- -99
      this.cell[,VARS[j]] <- -99
      this.cell[match(dates.avail, dates.thisyear), VARS[j]] <- vals
  }
  this.cell <- this.cell[,c("cell", "year", "doy", "srad", "tmin", "tmax", "vp", "wind", "prec")]
  chk.vals <- colMeans(this.cell[,c("srad", "tmin", "tmax", "vp", "wind", "prec")])
  if(sum(chk.vals==-99)==6) next
  to.file <- c(paste(t(c(xyFromCell(rst.aoi, idx[i]), round(elev[i,1],1), -99, -99)), collapse=", "),
                apply(this.cell, 1, paste, collapse=", "))

  writeLines(to.file, con=paste0(OUTPUT_DIR,"/nsch", cells[i,1], ".", substr(YEAR,2,4)))
  files.out <- c(files.out,paste0(OUTPUT_DIR,"/nsch", cells[i,1], ".", substr(YEAR,2,4)))
  progress <- i/length(idx)*100
  if(progress>=prog.x) {
      cat("=")
      prog.x <- prog.x + 2
  }
}
cat("| Done.\n")
zip(paste0(OUTPUT_DIR,"/../", paste(ISO3, YEAR, "15arcmin", sep="_"), ".zip"), files.out, flags= "-j -r9X")

cat("Download the output \n", paste0(OUTPUT_DIR,"/../", paste(ISO3, YEAR, "15arcmin", sep="_"), ".zip"))

# Debugging and Testing


---

This area is solely used for debugging. No need to run this section

In [ ]:
%%R

rst.elev_15 <- terra::aggregate(rst.elev, fact=3, na.rm=TRUE)

rst.base <- rast(rst.elev_15)
values(rst.base) <- 1:ncell(rst.base)
rst.base <- mask(rst.base, rst.elev_15)
rst.base <- crop(rst.base, vec_buff)

rst.aoi <- rasterize(vec_buff, rst.base)
rst.aoi <- crop(rst.aoi, rst.base)
rst.aoi <- mask(rst.aoi, rst.base)

rst.aoicells <- crop(rst.base, rst.aoi)
rst.aoicells <- mask(rst.aoicells, rst.aoi)
idx <- which(!is.na(values(rst.aoi)))
cells <- rst.aoicells[idx]

elev <- rst.elev_15[cells[,1]]
elev[is.na(elev)] <- -99

plot(rst.aoicells)

cat(paste(cells, collapse="\n"))